In [ ]:
import sys
sys.path.append(r'C:\codehome\eegdata\curry-python-reader')

import curryreader as cr

raw_data = cr.read(inputfilename =r'C:\codehome\eegdata\input\Acq 2025_05_23_1458.cdt', plotdata = 0, verbosity = 2)

import mne
import numpy as np

data = np.array(raw_data['data'])
ch_names = raw_data['labels']
sfreq = raw_data['info']['samplingfreq']

data_32 = data[:, :32].T
ch_names_32 = ch_names[:32]
print(ch_names_32)

info_32 = mne.create_info(ch_names=ch_names_32, sfreq=sfreq, ch_types='eeg')
montage = mne.channels.make_standard_montage('standard_1005')
info_32.set_montage(montage)

raw_32 = mne.io.RawArray(data_32, info_32)

raw_32.filter(l_freq=1, h_freq=50, fir_design='firwin')

from mne.preprocessing import ICA
ica = ICA(n_components=32)
ica.fit(raw_32)

In [ ]:
ica.exclude = [1, 4, 8]  
raw_clean = ica.apply(raw_32.copy())

In [ ]:
raw_clean.plot()

In [ ]:
import mne
import numpy as np
import pandas as pd

file_num = '1-1'


sfreq = raw_clean.info['sfreq']  # 用 raw_32 的取樣頻率

trigger_samples = [
    15500, 21900, 27000, 49000, 50500, 85500, 89000, 94000,
    124500, 187500, 208500, 275000, 303000, 357000, 385000, 421500, 
    475500, 641500, 650000, 716000, 733000, 742000, 766000, 771500, 
    950000, 926000
]


freq_bands = {
    'delta': (1, 4),
    'theta': (5, 7),
    'alpha': (8, 12),
    'beta': (13, 28),
    'gamma': (30, 50),
    'all': (1, 50)
}

channel_groups = {
    'frontal': [i-1 for i in [1,2,3,4,5,6,7,8,9,10,11]],
    'temporal_left': [i-1 for i in [7,12,17,23]],
    'temporal_right': [i-1 for i in [11,16,21,27]],
    'parietal': [i-1 for i in [18,19,20,21,22,24,25,26,27,28]],
    'occipital': [i-1 for i in [30,31,32]],
    'all': [i-1 for i in range(1,33)]
}

tmin, tmax = -2.0, 2.0

def get_freqs_idx(psd_freqs, fmin, fmax):
    idx_min = (np.abs(psd_freqs - fmin)).argmin()
    idx_max = (np.abs(psd_freqs - fmax)).argmin()
    if idx_min > idx_max:
        idx_min, idx_max = idx_max, idx_min
    return np.arange(idx_min, idx_max + 1)

results = []

for i, trig in enumerate(trigger_samples):
    if trig < 0 or trig > len(raw_clean.times):
        print(f"Warning: trigger {trig} out of data range, skipped")
        continue

    events = np.array([[trig, 0, 1]]) 
    epochs = mne.Epochs(raw_clean, events, event_id=1, tmin=tmin, tmax=tmax,
                        baseline=(None, 0), preload=True, verbose=False)

    pre_epochs = epochs.copy().crop(tmin, 0)
    post_epochs = epochs.copy().crop(0, tmax)

    pre_psd_obj = pre_epochs.compute_psd(fmin=1, fmax=50, n_fft=int(sfreq*2), n_overlap=0, method='welch', verbose=False)
    post_psd_obj = post_epochs.compute_psd(fmin=1, fmax=50, n_fft=int(sfreq*2), n_overlap=0, method='welch', verbose=False)

    pre_psd = pre_psd_obj.get_data()  # shape: (epochs, channels, freqs)
    post_psd = post_psd_obj.get_data()

  
    pre_psd_avg = pre_psd.mean(axis=0)  # (channels, freqs)
    post_psd_avg = post_psd.mean(axis=0)

    psd_freqs = pre_psd_obj.freqs

    for band, (fmin, fmax) in freq_bands.items():
        freqs_idx = get_freqs_idx(psd_freqs, fmin, fmax)
        for region, chans in channel_groups.items():
            pre_power = pre_psd_avg[np.ix_(chans, freqs_idx)].mean()
            post_power = post_psd_avg[np.ix_(chans, freqs_idx)].mean()
            power_change = post_power - pre_power
            results.append({
                'event': i+1,
                'band': band,
                'region': region,
                'pre_power': pre_power,
                'post_power': post_power,
                'power_change': power_change
            })

result_df = pd.DataFrame(results)
result_df.to_csv(f'C:\\codehome\\eegdata\\output\\{file_num}_power_p.csv', index=False)
print(result_df)


In [ ]:
import mne
import numpy as np
import pandas as pd

file_num = '6'

df = pd.read_csv(f'C:\\codehome\\eegdata\\output\\6\\{file_num}_clean_video_data.csv')

sfreq = raw_clean.info['sfreq']  # 直接用 raw_32 頻率

trigger_samples = [
    26495, 77006, 113377, 145503, 179004, 210038, 243137, 277568, 308209, 
    340253, 371991, 402917, 431338, 462183, 493992, 742129, 777861, 809845, 
    846827, 881015, 909346, 938910, 968842, 1003494, 1032189, 
    1073787, 1108062, 1140061, 1174161, 1206014, 1419019, 1454461, 1487205, 
    1514757, 1546663, 1582343, 1613804, 1646858, 1676821, 1706684, 1735093, 
    1766154, 1796956, 1830346, 1864866
]

trigger_times = np.array(trigger_samples) / sfreq

def calc_event_time(row):
    idx = int(row['playOrder']) - 1
    if idx < 0 or idx >= len(trigger_times):
        return np.nan
    return trigger_times[idx] + row['currentTime']

df['event_time'] = df.apply(calc_event_time, axis=1)
df = df.dropna(subset=['event_time'])
df['event_sample'] = (df['event_time'] * sfreq).astype(int)

events = np.column_stack((
    df['event_sample'].values,
    np.zeros(len(df), dtype=int),
    df['playOrder'].astype(int).values
))

tmin, tmax = -2.0, 2.0

event_id = {str(n): n for n in df['playOrder'].unique()}

epochs = mne.Epochs(raw_clean, events, event_id=event_id, tmin=tmin, tmax=tmax,
                    baseline=(None, 0), preload=True)

freq_bands = {
    'delta': (1, 4),
    'theta': (5, 7),
    'alpha': (8, 12),
    'beta': (13, 28),
    'gamma': (30, 50),
    'all': (1, 50)
}

channel_groups = {
    'frontal': [i-1 for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]],
    'temporal_left': [i-1 for i in [7, 12, 17, 23]],
    'temporal_right': [i-1 for i in [11, 16, 21, 27]],
    'parietal': [i-1 for i in [18,19,20,21,22,24,25,26,27,28]],
    'occipital': [i-1 for i in [30, 31, 32]],
    'all': [i-1 for i in range(1,33)]
}

def get_freqs_idx(psd_freqs, fmin, fmax):
    idx_min = (np.abs(psd_freqs - fmin)).argmin()
    idx_max = (np.abs(psd_freqs - fmax)).argmin()
    if idx_min > idx_max:
        idx_min, idx_max = idx_max, idx_min
    return np.arange(idx_min, idx_max + 1)

results = []

for event_key in event_id.keys():
    sel_epochs = epochs[event_key]

    pre_epochs = sel_epochs.copy().crop(tmin, 0)
    post_epochs = sel_epochs.copy().crop(0, tmax)

    pre_psd_obj = pre_epochs.compute_psd(fmin=1, fmax=50, n_fft=int(sfreq*2), n_overlap=0, method='welch')
    post_psd_obj = post_epochs.compute_psd(fmin=1, fmax=50, n_fft=int(sfreq*2), n_overlap=0, method='welch')

    pre_psd = pre_psd_obj.get_data()
    post_psd = post_psd_obj.get_data()

    pre_psd_avg = pre_psd.mean(axis=0)
    post_psd_avg = post_psd.mean(axis=0)

    psd_freqs = pre_psd_obj.freqs

    for band, (fmin, fmax) in freq_bands.items():
        freqs_idx = get_freqs_idx(psd_freqs, fmin, fmax)
        for region, chans in channel_groups.items():
            pre_power = pre_psd_avg[np.ix_(chans, freqs_idx)].mean()
            post_power = post_psd_avg[np.ix_(chans, freqs_idx)].mean()
            results.append({
                'event': event_key,
                'band': band,
                'region': region,
                'pre_power': pre_power,
                'post_power': post_power,
                'power_change': post_power - pre_power
            })

result_df = pd.DataFrame(results)
result_df.to_csv(f'C:\\codehome\\eegdata\\output\\{file_num}_power_n.csv', index=False)
print(result_df)


In [ ]:
from mne_icalabel import label_components

result = label_components(raw_32, ica, method='iclabel')

labels = result['labels']
probs = result['y_pred_proba']

for i, label in enumerate(labels[:20]):
    print(f"Component {i}: {label}, probability: {probs[i]:.3f}")

In [ ]:
for i, label in enumerate(labels[20:], start=20):
    print(f"Component {i}: {label}, probability: {probs[i]:.3f}")


In [ ]:
import matplotlib.pyplot as plt
ica.plot_components()


In [ ]:
ica.plot_sources(raw_32)

In [ ]:
ica.save('1_ica.fif')